In [ ]:

# Mongo Database credentials
t_host = "localhost"
t_port = "27017" # Mongo port
t_dbname = "testDB"
t_user = "mongoadmin"
t_pw = "[Mongo database user password here]"
client = MongoClient("mongodb://" + t_user + ":" + t_pw + "@" + t_host + ":" + t_port + "/" + t_dbname + "/?tls=true")
db_mongo = client.testDB
users_mongo = db_mongo.users

# Postgres Database credentials
t_host = "[database address here]"
t_port = "5432" # Default postgres port
t_dbname = "[database name here]"
t_user = "[database user name here]"
t_pw = "[database user password here]"
db_conn = psycopg2.connect(host=t_host, port=t_port, dbname=t_dbname, user=t_user, password=t_pw)
db_cursor = db_conn.cursor()

def migrate():
    # Build array_rows here from Mongo users collection
    array_users_name = []
    array_users_age = []

    # Pull all documents (records) from users collection (table)
    mongo_results = users_mongo.find()
    for record in mongo_results:
        array_users_name.append(record['name'])
        array_users_age.append(record['age'])

    # Data validation would go here.

    # Cleanup our database collection connection
    mongo_results.close()

    # SEND rows to the Postgres database
    s = ""
    s += "INSERT INTO tbl_users "
    s += "("
    s += "t_name"
    s += ", i_age"
    s += ")"
    s += " VALUES"
    s += "("
    s += "'" + array_users_name + "'"
    s += "," + array_users_age
    s += ")"
    # IMPORTANT WARNING: this format allows for a user to try to insert
    #   potentially damaging code, commonly known as "SQL injection".
    #   You will want to investigate methods for preventing this, including
    #   use of stored procedures.

    # Here we are catching and displaying any errors that occur
    #   while TRYing to commit the execution of our SQL script.
    db_cursor.execute(s)
        try:
            db_conn.commit()
        except psycopg2.Error as e:
            # Create a message to send to the user with valuable information, including
            #    the error message and the SQL that caused the error.
            t_message = "Database error: " + e + "/n SQL: " + s
            # Notice: you may want to create a template page called "error.html" for handling errors.
            #   We are also sending a parameter along with our call to the page.
            return render_template("error.html", message = t_message)

        # They got this far, meaning success
        # Clean up our Postgres database connection and cursor object
        db_cursor.close()
        db_conn.close()

    # Redirect user to the rest of your application
    return redirect("http://your-URL-here", code=302)

if __name__ == "__migrate__":
    migrate()